In [2]:
import lxml.html as lx
import pandas as pd
import numpy as np
import requests
import requests_cache
import numpy as np
import nltk
import nltk.corpus
from nltk.corpus import wordnet
from itertools import chain
from plotnine import *
import time
import matplotlib.pyplot as plt
import argparse
%matplotlib inline

In [5]:
#OCE forum
def get_page_url(base_url, page_url):
    requests_cache.install_cache('page_logs')
    next_page_available = True
    cur_page = 1;
    urls = []
    while True:
        try:
            time.sleep(.1)
            response = requests.get(page_url+str(cur_page))
            html = lx.fromstring(response.text)
            html.make_links_absolute(base_url)
            url = html.xpath("//tbody//td/a/@href")
            urls += url
            cur_page +=1;
        except:
            break
    return urls

base_url = "http://forums.oce.leagueoflegends.com/"
extension = "board/forumdisplay.php?f=59&page="
pages_url = get_page_url(base_url, base_url+extension)

In [7]:
len(pages_url)

73043

In [8]:
def unique(links):
    url = []
    for items in links:
        if items not in url:
            url.append(items)
    return url
pages_url = unique(pages_url)
pages_url

['http://forums.oce.leagueoflegends.com/board/showthread.php?t=87067',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=51722',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=87020',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=87972',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=75662',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=76686',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=80419',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=88406',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=88407',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=88403',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=88304',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=88386',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=88387',
 'http://forums.oce.leagueoflegends.com/board/showthread.php?t=88391',
 'http

In [9]:
op_url = "https://oce.op.gg/summoner/userName="

def rank_lvl(name):
    requests_cache.install_cache("opna")
    time.sleep(.1)
    response = requests.get(op_url+name)
    response.raise_for_status()
    html = lx.fromstring(response.text)
    
    lvl = html.xpath("//*[contains(@class,'Level tip')]")
    recent_rank = html.xpath("//*[contains(@class, 'PastRankList')]/li[last()]")
    if len(lvl) > 0:
        lvl = lvl[0].text_content().strip()
    
    if len(recent_rank) > 0:
        recent_rank = recent_rank[0].text_content().strip()
        
    if len(lvl) > 0 and len(recent_rank) >0:
        return recent_rank,lvl
    
    elif len(lvl)>0:
        return np.nan,lvl
    
    else:
        return np.nan, np.nan

In [ ]:
def page_info(url):
    dates = []
    names = []
    members = []
    texts = []
    lvl = []
    rank = []
    requests_cache.install_cache("page_posts")
    try:
        response = requests.get(url)
        response.raise_for_status()
        html = lx.fromstring(response.text)
        time.sleep(.1)
        title = html.xpath("//*[contains(@class, 'float-left')]/h3")[0].text_content()
        all_dates = html.xpath("//*[contains(@class, 'post-header padding-side-10 clearfix')]")
        all_users = html.xpath("//*[contains(@class, 'post-user')]")
        member_type = html.xpath("//*[contains(@class, 'post-col-left')]//p")[1::2]
        for i in range(len(all_dates)):
            dates.append(all_dates[i].text_content().strip())
            names.append(all_users[i].text_content().strip())
            members.append(member_type[i].text_content().strip())
            texts.append(html.xpath("//*[contains(@class,'post-message')]//p")[i].text_content().strip())
        for name in names:
            op_na = rank_lvl(name)
            rank.append(op_na[0])
            lvl.append(op_na[1])
        return pd.DataFrame({'user_name':names,"tier": rank, "level": lvl, "tier": rank, "level": lvl, 'member_type': members, 'dates': dates,'title': title, 'text':texts})
    except:
        return

eu_page_df = [page_info(url) for url in pages_url]